In [1]:
#  ▄▄▄▄    ▄▄▄       ███▄    █  ▄████▄   ▒█████   ██▓     ▒█████   ███▄ ▄███▓ ▄▄▄▄    ██▓ ▄▄▄      
# ▓█████▄ ▒████▄     ██ ▀█   █ ▒██▀ ▀█  ▒██▒  ██▒▓██▒    ▒██▒  ██▒▓██▒▀█▀ ██▒▓█████▄ ▓██▒▒████▄    
# ▒██▒ ▄██▒██  ▀█▄  ▓██  ▀█ ██▒▒▓█    ▄ ▒██░  ██▒▒██░    ▒██░  ██▒▓██    ▓██░▒██▒ ▄██▒██▒▒██  ▀█▄  
# ▒██░█▀  ░██▄▄▄▄██ ▓██▒  ▐▌██▒▒▓▓▄ ▄██▒▒██   ██░▒██░    ▒██   ██░▒██    ▒██ ▒██░█▀  ░██░░██▄▄▄▄██ 
# ░▓█  ▀█▓ ▓█   ▓██▒▒██░   ▓██░▒ ▓███▀ ░░ ████▓▒░░██████▒░ ████▓▒░▒██▒   ░██▒░▓█  ▀█▓░██░ ▓█   ▓██▒
# ░▒▓███▀▒ ▒▒   ▓▒█░░ ▒░   ▒ ▒ ░ ░▒ ▒  ░░ ▒░▒░▒░ ░ ▒░▓  ░░ ▒░▒░▒░ ░ ▒░   ░  ░░▒▓███▀▒░▓   ▒▒   ▓▒█░
# ▒░▒   ░   ▒   ▒▒ ░░ ░░   ░ ▒░  ░  ▒     ░ ▒ ▒░ ░ ░ ▒  ░  ░ ▒ ▒░ ░  ░      ░▒░▒   ░  ▒ ░  ▒   ▒▒ ░
#  ░    ░   ░   ▒      ░   ░ ░ ░        ░ ░ ░ ▒    ░ ░   ░ ░ ░ ▒  ░      ░    ░    ░  ▒ ░  ░   ▒   
#  ░            ░  ░         ░ ░ ░          ░ ░      ░  ░    ░ ░         ░    ░       ░        ░  ░
#       ░                      ░                                                   ░               

In [2]:
import pandas as pd

In [3]:
# Reading the file with specified encoding.
df = pd.read_csv('/Users/alejandro/Desktop/Life/Apps/Card-Fraud-Detection/cfd/dataset.csv', encoding='utf-8')
# Printing the first few rows.
print(df.head())

                                                 URL
0         ftp://188.128.111.33/IPTV/TV1324/view.html
1                   ftp://188.128.111.33/web/sec.htm
2  ftp://me@createkindlebooks.org:Noobasshole@cre...
3  http://000000000000000000000000000000000000000...
4  http://00000000000000000000000000000000000000d...
